In [2]:
from Bio import SeqIO

# Bloque 1 - Descarga de secuencia desde UniProt

En este apartado, se lleva a cabo la descarga de la proteína **Beta-2-microglobulin (B2M)** desde UniProt. Además, se verifica su contenido mostrando por pantalla la longitud total y los primeros 20 aminoácidos de la secuencia.

In [3]:
fasta_path = "data/P61769.fasta.txt"

record = SeqIO.read(fasta_path, "fasta")

sequence = record.seq

print("ID de la secuencia:", record.id)
print("Descripción:", record.description)
print("Longitud total de la secuencia:", len(sequence))
print("Primeros 20 aminoácidos:", sequence[:20])

ID de la secuencia: sp|P61769|B2MG_HUMAN
Descripción: sp|P61769|B2MG_HUMAN Beta-2-microglobulin OS=Homo sapiens OX=9606 GN=B2M PE=1 SV=1
Longitud total de la secuencia: 119
Primeros 20 aminoácidos: MSRSVALAVLALLSLSGLEA
